### [Hierarchical AR(K) Models](https://discourse.mc-stan.org/t/hierarchical-ar-k-models/781)

In [1]:
#load libraries
library(rstan)
library(coda)
library(dplyr)

Loading required package: ggplot2
Loading required package: StanHeaders
rstan (Version 2.18.2, GitRev: 2e1f913d3ca3)
For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)

Attaching package: ‘coda’

The following object is masked from ‘package:rstan’:

    traceplot


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



```r
data {
  int<lower=1> N; // total number of observations
  int<lower=1> K; // number of lagged factor
  int<lower=1> numG; // number of groups
  vector[N] y; // data for Hierarchical AR(K)
  int sizes[numG]; // sizes of observations across groups
}
parameters {
  real<lower = 0> local_sigma; 
  vector<lower = 0>[K] global_sigma;
  vector[K] z[numG];
  vector<lower = -2, upper = 2>[K] global_beta; // global AR(K) multipliers
}
transformed parameters {
  vector[K] group_betas[numG]; // group level AR(K) multipliers
  for(i in 1:numG) {
    group_betas[i] = global_beta + global_sigma .* z[i]; // NCP formulation, no correlations
  }
}
model {
  int pos;
  pos = 1;
  for(i in 1:numG) {  // loop over groups
    int local_N;
    vector[sizes[i]] local_y;
    local_N = sizes[i];
    local_y = segment(y, pos, local_N);
    for (n in (K + 1):local_N) { // loop over observations
      real mu;
      mu = 0;
      for (k in 1:K) { // loop over lags
        mu = mu + group_betas[i][k] * local_y[n - k];
      }
      y[n] ~ normal(mu, local_sigma);
    }
    z[i] ~ normal(0, 1);
    pos = pos + local_N;
  }
  
  // hyperpriors
  local_sigma ~ cauchy(0, 2);
  global_sigma ~ cauchy(0, 2);
  global_beta ~ cauchy(0, 2);
}
```

In [30]:
df_eu  <-  read.csv(file="../disaggregated_ts/hts_eu_a10.csv", header=TRUE, sep=",", row.names =1 )
df_eu  <-  df_eu[ , order(names(df_eu))]
dat  <-  df_eu[,1:30]

In [47]:
dim(dat)

[1] 75 30

In [44]:
for (i in colnames(dat)){
    cat(i, '+', 'lag(',i,') + ')}

ATA + lag( ATA ) + ATB + lag( ATB ) + ATF + lag( ATF ) + ATG + lag( ATG ) + ATJ + lag( ATJ ) + ATK + lag( ATK ) + ATL + lag( ATL ) + ATM + lag( ATM ) + ATO + lag( ATO ) + ATR + lag( ATR ) + BEA + lag( BEA ) + BEB + lag( BEB ) + BEF + lag( BEF ) + BEG + lag( BEG ) + BEJ + lag( BEJ ) + BEK + lag( BEK ) + BEL + lag( BEL ) + BEM + lag( BEM ) + BEO + lag( BEO ) + BER + lag( BER ) + BGA + lag( BGA ) + BGB + lag( BGB ) + BGF + lag( BGF ) + BGG + lag( BGG ) + BGJ + lag( BGJ ) + BGK + lag( BGK ) + BGL + lag( BGL ) + BGM + lag( BGM ) + BGO + lag( BGO ) + BGR + lag( BGR ) + 

In [45]:
#the model
X<-model.matrix(~ATA + lag( ATA ) + ATB + lag( ATB ) + ATF + lag( ATF ) + ATG + lag( ATG ) + ATJ + lag( ATJ ) + ATK + lag( ATK ) + ATL + lag( ATL ) + ATM + lag( ATM ) + ATO + lag( ATO ) + ATR + lag( ATR ) + BEA + lag( BEA ) + BEB + lag( BEB ) + BEF + lag( BEF ) + BEG + lag( BEG ) + BEJ + lag( BEJ ) + BEK + lag( BEK ) + BEL + lag( BEL ) + BEM + lag( BEM ) + BEO + lag( BEO ) + BER + lag( BER ) + BGA + lag( BGA ) + BGB + lag( BGB ) + BGF + lag( BGF ) + BGG + lag( BGG ) + BGJ + lag( BGJ ) + BGK + lag( BGK ) + BGL + lag( BGL ) + BGM + lag( BGM ) + BGO + lag( BGO ) + BGR + lag( BGR ),dat)
head(X)

,(Intercept),ATA,lag(ATA),ATB,lag(ATB),ATF,lag(ATF),ATG,lag(ATG),ATJ,⋯,BGK,lag(BGK),BGL,lag(BGL),BGM,lag(BGM),BGO,lag(BGO),BGR,lag(BGR)
2000-04-01,1,965.10,831.80,11447.50,11055.15,3603.20,3068.05,10724.70,10558.90,1523.85,⋯,69.95,70.25,357.35,334.70,129.65,104.45,493.50,493.65,67.00,63.70
2000-07-01,1,883.70,965.10,11531.30,11447.50,3705.15,3603.20,10927.95,10724.70,1541.60,⋯,77.05,69.95,364.45,357.35,147.05,129.65,528.00,493.50,63.75,67.00
2000-10-01,1,846.95,883.70,12257.60,11531.30,3941.00,3705.15,11148.90,10927.95,1658.85,⋯,82.60,77.05,396.10,364.45,160.15,147.05,632.15,528.00,77.10,63.75
2001-01-01,1,883.40,846.95,12011.20,12257.60,3046.85,3941.00,11107.70,11148.90,1724.85,⋯,83.85,82.60,418.30,396.10,117.65,160.15,512.20,632.15,61.10,77.10
2001-04-01,1,1064.60,883.40,12075.80,12011.20,3491.15,3046.85,10965.85,11107.70,1763.55,⋯,93.45,83.85,318.05,418.30,132.25,117.65,532.40,512.20,65.55,61.10
2001-07-01,1,917.45,1064.60,11993.55,12075.80,3698.35,3491.15,11059.70,10965.85,1836.95,⋯,105.40,93.45,374.60,318.05,181.80,132.25,525.15,532.40,58.95,65.55


In [ ]:
#the regression slopes
betas<-runif(4,-1,1)
betas
#the standard deviation for the simulated data
sigma<-1

#the simulated data
y_norm<-rnorm(100,X%*%betas,sigma)
y_norm


#a matrix to get the predicted y values
new_X<-model.matrix(~x1*x2,expand.grid(x1=seq(min(dat$x1),max(dat$x1),length=20),x2=c(min(dat$x2),mean(dat$x2),max(dat$x2))))
new_X  %>%  head()

In [46]:
#the location of the model files
setwd("/home/xenakas/Desktop/Git/bayesian_inference/STAN")

In [55]:
dim(dat)

[1] 75 30

In [63]:
start_time <- Sys.time()
#the model
m_norm<-stan(file="hm.stan",data = list(N=30,K=4,numG=3,y=dat,sizes= c(10,10,10)),pars = c("local_sigma","global_sigma","z", "global_beta"))
end_time <- Sys.time()
end_time - start_time

failed to create the sampler; sampling not done


Time difference of 0.05053973 secs

In [59]:
75*3

[1] 225

In [62]:
dat[1]

,ATA
2000-01-01,831.80
2000-04-01,965.10
2000-07-01,883.70
2000-10-01,846.95
2001-01-01,883.40
2001-04-01,1064.60
2001-07-01,917.45
2001-10-01,813.85
2002-01-01,887.35
2002-04-01,948.60
